In [462]:
fname = 'CSW_corpus.txt'
with open(fname) as file:
    corpus = file.readlines()
corpus = [x.strip() for x in corpus]   
# delete first 2 rows
corpus = corpus[2:]
# de-capitalize
corpus = [x.lower() for x in corpus]

In [463]:
import numpy as np
import pandas as pd
from collections import Counter 
from nltk import edit_distance

def typo_classification(typo,correct):
    if (len(typo) > len(correct)):
        return 'insertion'
    elif (len(typo) < len(correct)):
        return 'deletion'
    else:
        typo_count = Counter(typo)
        correct_count = Counter(correct)
        if typo_count == correct_count:
            return 'reversal'
        else:
            return 'subsititution'

def find_candidates(typo,corpus):
    candidates = []
    candi_type = []
    for word in corpus:
        ed = edit_distance(typo,word)
        word_type = typo_classification(typo,word)
        if ((ed == 1) | ((ed == 2) & (word_type == 'reversal'))):
            candidates.append(word)
            candi_type.append(word_type)
    return candidates,candi_type

def find_position(typo,candidates):
    position = []
    for corr in candidates:
        typo_type = typo_classification(typo,corr)
        
        if (typo_type == 'deletion'):
            typo += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if corr[i] != corr[i-1]:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        break
                    else:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        position.append([typo,corr,"@",corr[i],i-1,typo_type])
                        break
                        
                i += 1
        elif (typo_type == 'insertion'):
            corr += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if typo[i] != typo[i-1]:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        break
                    elif ((typo[i] == typo[i-1])& (typo[i] == typo[i-2])):
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        position.append([typo,corr,typo[i],"@",i-2,typo_type])
                        break
                    else:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        break
                i += 1
        elif (typo_type == 'subsititution'):
            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    position.append([typo,corr,typo[i],corr[i],i,typo_type])
                    break
                i+=1
                
        elif (typo_type == 'reversal'):
            i = 0
            while i < len(corr)-1:
                if ((typo[i] == corr[i+1]) & (typo[i+1] == corr[i])):
                    typo_comb = typo[i] + typo[i+1]
                    position.append([typo,corr,typo_comb,typo_comb[::-1],i,typo_type])
                    break
                i +=1
    return position

In [464]:
# imput lowercase typo
typo = 'ambitios'

candidates,cand_type = find_candidates(typo,corpus)
correction = find_position(typo,candidates)

In [461]:
correction = pd.DataFrame(correction)
correction.columns = ['Typo','Correction','old','new','index','type']
correction

,Typo,Correction,old,new,index,type
0,ambitios,ambition,s,n,7,subsititution
1,ambitios,ambitions,@,n,7,deletion
2,ambitios,ambitious,@,u,7,deletion
